## Imports

In [1]:
import copy
import numpy as np
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from config import config
import os
import sys
current_dir = os.getcwd()
path = "C:\\Users\\eirik\\Documents\\Master\\ISLBBNN\\islbbnn"
# path = "C:\\you\\path\\to\\islbbnn\\folder\\here"
os.chdir(path)
import plot_functions as pf
import pipeline_functions as pip_func
sys.path.append('networks')
from lrt_sigmoid_net import BayesianNetwork

os.chdir(current_dir) # set the working directory back to this one 

# Attain data

Problem:

$$y = x_1 + x_2 + 100 +\epsilon$$

where $\epsilon \sim N(0,0.01)$. 


Can make $x_3$ dependent on $x_1$. The depedence is defined in the following way:

\begin{align*}
 x_1 &\sim Unif(-10,10) \\
 x_3 &\sim Unif(-10,10) \\
 x_3 &= \text{dep}\cdot x_1 + (1-\text{dep})\cdot x_3
\end{align*}

## Pre process and batch size

In [2]:
# define parameters
HIDDEN_LAYERS = config['n_layers'] - 2 
epochs = config['num_epochs']
post_train_epochs = config['post_train_epochs']
dim = config['hidden_dim']
n_nets = config['n_nets']
n_samples = config['n_samples']
lr = config['lr']
class_problem = config["class_problem"]
non_lin = config["non_lin"]
verbose = config['verbose']
save_res = config['save_res']
patience = config['patience']
SAMPLES = 1


# Get linear data, here a regression problem
y, X = pip_func.create_data_unif(n_samples, beta=[100,1,1,1,1], dep_level=0.0, classification=class_problem)

n, p = X.shape  # need this to get p 
print(n,p,dim)

# Define BATCH sizes
BATCH_SIZE = int((n*0.8)/100)
TEST_BATCH_SIZE = int(n*0.10) # Would normally call this the "validation" part (will be used during training)
VAL_BATCH_SIZE = int(n*0.10) # and this the "test" part (will be used after training)

TRAIN_SIZE = int((n*0.80)/100)
TEST_SIZE = int(n*0.10) # Would normally call this the "validation" part (will be used during training)
VAL_SIZE = int(n*0.10) # and this the "test" part (will be used after training)

NUM_BATCHES = TRAIN_SIZE/BATCH_SIZE

print(NUM_BATCHES)

assert (TRAIN_SIZE % BATCH_SIZE) == 0
assert (TEST_SIZE % TEST_BATCH_SIZE) == 0

40000 4 10
1.0


## Seperate a test set for later

In [3]:
# Split keep some of the data for validation after training
X, X_test, y, y_test = train_test_split(
    X, y, test_size=0.10, random_state=42)#, stratify=y)

test_dat = torch.tensor(np.column_stack((X_test,y_test)),dtype = torch.float32)

# Train network

## Device setup

In [4]:
# select the device and initiate model

# DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "mps")
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
LOADER_KWARGS = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}

## Train, validate and test network

In [5]:
all_nets = {}
metrics_several_runs = []
metrics_median_several_runs = []
for ni in range(n_nets):
    post_train = False
    print('network', ni)
    # Initate network
    torch.manual_seed(ni+42)
    net = BayesianNetwork(dim, p, HIDDEN_LAYERS, classification=class_problem).to(DEVICE)
    alphas = pip_func.get_alphas_numpy(net)
    nr_weights = np.sum([np.prod(a.shape) for a in alphas])
    print(nr_weights)

    optimizer = optim.Adam(net.parameters(), lr=lr)
    
    all_nll = []
    all_loss = []

    # Split into training and test set
    X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=1/9, random_state=ni)#, stratify=y)
            
    train_dat = torch.tensor(np.column_stack((X_train,y_train)),dtype = torch.float32)
    val_dat = torch.tensor(np.column_stack((X_val,y_val)),dtype = torch.float32)
    
    # Train network
    counter = 0
    highest_acc = 0
    best_model = copy.deepcopy(net)
    for epoch in range(epochs + post_train_epochs):
        if verbose:
            print(epoch)
        nll, loss = pip_func.train(net, train_dat, optimizer, BATCH_SIZE, NUM_BATCHES, p, DEVICE, nr_weights, post_train=post_train)
        nll_val, loss_val, ensemble_val = pip_func.val(net, val_dat, DEVICE, verbose=verbose, reg=(not class_problem))
        if ensemble_val >= highest_acc:
            counter = 0
            highest_acc = ensemble_val
            best_model = copy.deepcopy(net)
        else:
            counter += 1
        
        all_nll.append(nll)
        all_loss.append(loss)

        if epoch == epochs-1:
            post_train = True   # Post-train --> use median model 
            for name, param in net.named_parameters():
                for i in range(HIDDEN_LAYERS+1):
                    #if f"linears{i}.lambdal" in name:
                    if f"linears.{i}.lambdal" in name:
                        param.requires_grad_(False)

        if counter >= patience:
            break
        
    all_nets[ni] = net 
    # Results
    metrics, metrics_median = pip_func.test_ensemble(all_nets[ni], test_dat, DEVICE, SAMPLES=10, reg=(not class_problem)) # Test same data 10 times to get average 
    metrics_several_runs.append(metrics)
    metrics_median_several_runs.append(metrics_median)
    pf.run_path_graph(all_nets[ni], threshold=0.5, save_path=f"path_graphs/lrt/prob/test{ni}", show=verbose)

if verbose:
    print(metrics)
m = np.array(metrics_several_runs)
m_median = np.array(metrics_median_several_runs)

network 0
54
0
loss 352.1375427246094
nll 16.846179962158203
density 0.7535007612572776

val_loss: 605.5336, val_nll: 258.7996, val_ensemble: 0.9818, used_weights_median: 39

1
loss 80.17396545410156
nll 21.203083038330078
density 0.20620262129577221

val_loss: 231.7242, val_nll: 174.6580, val_ensemble: 0.9882, used_weights_median: 7

2
loss 52.81830596923828
nll 12.415382385253906
density 0.13244875586005272

val_loss: 163.1057, val_nll: 123.2184, val_ensemble: 0.9952, used_weights_median: 6

3
loss 49.68548583984375
nll 9.130159378051758
density 0.1271029103167907

val_loss: 141.2609, val_nll: 100.6481, val_ensemble: 0.9985, used_weights_median: 6

4
loss 50.45975875854492
nll 10.2677001953125
density 0.12472027431552608

val_loss: 133.8495, val_nll: 93.3276, val_ensemble: 0.9968, used_weights_median: 6

5
loss 46.809234619140625
nll 11.675315856933594
density 0.10539312931467537

val_loss: 121.8381, val_nll: 86.3489, val_ensemble: 0.9970, used_weights_median: 5

6
loss 41.5405120849

TODO: Seems like val_nll and val_loss is the same, but this is not true. Fix this...

## Function for plotting weight magnitude

In [6]:
pf.run_path_graph_weight(net, save_path="path_graphs/lrt/weight/temp", show=True)

In [7]:
pip_func.get_alphas_numpy(net)

[array([[0.04988745, 0.04988742, 0.04988135, 0.02062172],
        [0.04685967, 0.04959689, 0.04991695, 0.04983602],
        [0.0498042 , 0.047339  , 0.04862013, 0.0498325 ],
        [0.04926866, 0.04913945, 0.0495112 , 0.04981361],
        [0.04986336, 0.02397774, 0.04915522, 0.04956369],
        [0.9999999 , 0.99998546, 0.04150493, 0.02968235],
        [0.04990653, 0.04990479, 0.04930333, 0.04955413],
        [0.04664429, 0.0493351 , 0.04989627, 0.04982524],
        [0.04990931, 0.04990511, 0.04989426, 0.04989322],
        [0.01056166, 0.04988218, 0.04983991, 0.04980685]], dtype=float32),
 array([[1.7489504e-02, 1.7768953e-02, 8.5350974e-03, 1.8286632e-02,
         1.5456454e-02, 9.6225744e-04, 1.7738851e-02, 1.7712986e-02,
         1.7755657e-02, 1.7462607e-02, 1.0000000e+00, 9.9999976e-01,
         2.3380416e-03, 2.2668489e-03]], dtype=float32)]

In [8]:
pip_func.weight_matrices_numpy(net)

[array([[ 3.3769492e-02,  1.2903831e-02,  4.7956183e-04,  3.1975642e-01],
        [ 7.1882433e-01,  2.3529282e-01,  2.3785027e-02,  7.2391443e-02],
        [ 1.1623137e-01,  5.7366705e-01,  7.0812022e-03, -1.7857222e-02],
        [-1.5557915e-01, -8.0238491e-02,  2.4338402e-01, -9.3940124e-02],
        [ 2.8213667e-02, -9.7034678e-02, -2.8946355e-01, -1.9545349e-01],
        [-1.7739824e-08, -1.9720712e-08,  2.2152138e-03,  1.4615100e-03],
        [-3.3941165e-02, -2.0993117e-02, -2.8660262e-01, -8.4736086e-02],
        [-7.2170126e-01, -2.9008219e-01,  1.2678406e-02,  6.1826140e-02],
        [ 1.2690320e-02,  4.7414652e-03, -5.2098912e-04,  2.2329235e-02],
        [-9.9892294e-01,  3.0638261e-03, -5.8946375e-02,  6.2813371e-02]],
       dtype=float32),
 array([[ 1.16950594e-01,  2.41647810e-01,  8.45124900e-01,
          1.73156746e-02,  1.65198624e-01, -1.84700561e+00,
         -7.16073960e-02, -1.26342952e-01,  6.96365163e-02,
         -1.35228261e-01,  4.00830889e+00,  4.03862238e+